In [ ]:
import numpy as np
import math
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')

In [ ]:
#Loading the data
data = sc.textFile('/FileStore/tables/plot_summaries.txt')
metadata = sc.textFile('/FileStore/tables/movie_metadata.tsv')

In [ ]:
# Get movie_id, movie_name from metadata
metadata = metadata.map(lambda x: x.split('\t')).map(lambda x: (x[0], x[2]))
 
#Find total #movies
tot_docs = data.count()

In [ ]:
def tokens_no_stop_words(txt):
    stop_words = set(stopwords.words('english'))
    return [x.lower() for x in nltk.word_tokenize(txt) if x.lower() not in stop_words and len(x) > 2]
 
def word_docid_1(docid, words):
    tmp = [((x,docid),1) for x in words]
    return tmp
 
def cal_ifidf(lst):
    return (lst[0]/lst[1])*math.log(tot_docs/lst[2])
  
def cal_idf(lst):
    return math.log(tot_docs/lst[2])
 
def cosine_similarity(query, data):
    numerator = { k: [0]*len(query) for k in all_movies}
    denom =  { k: [0]*len(query) for k in all_movies}
    init_d = [np.square(x) for x in query.values()]
 
    for i, wrd in enumerate(query.keys()):
        tmp=query[wrd]
        for doc_id,v in data[wrd]:
            numerator[doc_id][i]= v*tmp
            denom[doc_id][i] = np.square(v)
    
    for doc, lst in denom.items():
        lst.extend(init_d)
        denom[doc] = np.sqrt(sum(lst))
    numerator = {k: sum(v) for k,v in numerator.items()}
    res = {}
    for k,v in numerator.items():
        res[k] = v/denom[k]
 
    return sorted(res.items(), key=lambda x: x[1], reverse=True)[:10]

In [ ]:
# CALCULATING WORD COUNT
 
# (doc_id,[words...])
data_woth_no_stop_words = data.map(lambda x: x.split('\t')).map(lambda x: (x[0],tokens_no_stop_words(x[1])))
# ((word, doc_id), 1)
map1 = data_woth_no_stop_words.flatMap(lambda x: (((i,x[0]),1) for i in x[1]))
# (doc_id, wrds_per_doc)
words_per_doc = data_woth_no_stop_words.map(lambda x: (x[0] ,len(x[1])))
all_movies = words_per_doc.map(lambda x: x[0]).collect()
# ((word, doc_id), wrd_count)
reduce1 = map1.reduceByKey(lambda x,y: x+y)

In [ ]:
# CALCULATE WORDCOUNT PER DOCUMENT
 
# ((word, doc_id), wrd_count) to (doc_id, (word, wrd_count))
map2 = reduce1.map(lambda x: (x[0][1],(x[0][0],x[1])))
# (doc_id, (word, wrd_count)) to (doc_id, [(word, wrd_count)], wrds_per_doc)
map2 = map2.groupByKey().mapValues(list)
reduce2 = map2.join(words_per_doc)
#(doc_id, [(word, wrd_count)], wrds_per_doc) to ((word, doc_id), (wrd_count, wrds_per_doc))
map3 = reduce2.flatMap(lambda x: (((y[0], x[0]),(y[1], x[1][1])) for y in x[1][0]))

In [ ]:
# CALCULATE DOC COUNT PER WORD
 
#((word, doc_id), (wrd_count, wrds_per_doc)) to (word, [doc_id, wrd_count, wrds_per_doc])
map3 = map3.map(lambda x: (x[0][0], [x[0][1], x[1][0], x[1][1]]))
# (word, [doc_id, wrd_count, wrds_per_doc]) to (word, [[doc1, wrd_count1, wrds_per_doc1]])
map3 = map3.groupByKey().mapValues(list)
# (word, [[doc1, wrd_count1, wrds_per_doc1]]) to ((word, doc1), [wrd_count1, wrds_per_doc1, docs_per_word])
map3 =map3.flatMap(lambda x: (((x[0], entry[0]), [entry[1], entry[2], len(x[1])]) for entry in x[1] ))

In [ ]:
# CALCULATE IDF
# ((word, doc1), [wrd_count1, wrds_per_doc1, docs_per_word]) to (word, tf)
#tf = map3.map(lambda x: (x[0], cal_tf(x[1])))
idf = map3.map(lambda x: (x[0][0], cal_idf(x[1])))
idf = idf.distinct()
 
# CALCULATE TF-IDF
# ((word, doc1), [wrd_count1, wrds_per_doc1, docs_per_word]) to ((word,doc_id), tfidf)
tfidf = map3.map(lambda x: (x[0], cal_ifidf(x[1])))

In [ ]:
#                    SINGLE WORD SEARCH 
 
#Enter the word to search below and execute the cell.
search_word = 'katniss'
 
###############################################################
 
res = tfidf.filter(lambda x: x[0][0] == search_word.lower()).sortBy(lambda x: -x[1]).map(lambda x: (x[0][1], x[1]))
res = res.join(metadata).map(lambda x: (x[1][1]))
res.take(10)